# 02 - Measuring Investment Risk 

<b>Quantifying risk :</b>
These can help:

  
- Standard deviation
  - Standard deviation is a measure of how much an investment's returns can vary from its average return. It is a measure of volatility and in turn, risk (ref: https://investinganswers.com/dictionary/s/standard-deviation) 
  - S = [1/n *  (ri - rave)2]½
    - where:
    - ri = actual rate of return
    - rave = average rate of return
    - n = number of time periods

- Variance : 
  - High variance in a stock is associated with higher risk, along with a higher return. Low variance is associated with lower risk and a lower return. High variance stocks tend to be good for aggressive investors who are less risk-averse, while low variance stocks tend to be good for conservative investors who have less risk tolerance (ref: https://www.investopedia.com/ask/answers/041415/variance-good-or-bad-stock-investors.asp) 
  - Variance = $S^2$

## Calculating Risk of a Security

In [9]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt

startDate = '2015-01-01'
endDate = '2019-12-31'

tickers = ['MSFT', 'IBM']
securityData = pd.DataFrame()

for t in tickers:
    securityData[t] = wb.DataReader(t, data_source='yahoo', start=startDate, end=endDate)['Adj Close']


In [4]:
securityData.tail()

,MSFT,IBM
Date,,
2020-07-22,211.750000,128.669998
2020-07-23,202.539993,127.330002
2020-07-24,201.300003,125.790001
2020-07-27,203.850006,126.209999
2020-07-28,202.020004,124.470001


In [11]:
securityReturns = np.log(securityData / securityData.shift(1))
securityReturns

,MSFT,IBM
Date,,
2015-01-02,NaN,NaN
2015-01-05,-0.009238,-0.015860
2015-01-06,-0.014786,-0.021802
2015-01-07,0.012625,-0.006557
2015-01-08,0.028994,0.021502
...,...,...
2019-12-24,-0.000191,-0.004214
2019-12-26,0.008163,-0.000519
2019-12-27,0.001826,0.002665


### MSFT 

In [30]:
msftMeanDaily = securityReturns['MSFT'].mean() # Daily average return
msftMeanAnnually = securityReturns['MSFT'].mean() * 250 # yearly average return
msftStandardDeviationDaily = securityReturns['MSFT'].std() # standard deviation for day
msftStandardDeviationAnnually = securityReturns['MSFT'].std() * 250 ** 0.5 # standard variation is square root of the variance

### IBM

In [31]:
ibmMeanDaily = securityReturns['IBM'].mean() # Daily average return
ibmMeanAnnually = securityReturns['IBM'].mean() * 250 # yearly average return
ibmStandardDeviationDaily = securityReturns['IBM'].std() # standard deviation for day
ibmStandardDeviationAnnually = securityReturns['IBM'].std() * 250 ** 0.5 # standard variation is square root of the variance

### Compare MSFT vs IBM

In [46]:
print('MEANS --')
print('MSFT: %f - IBM: %f' %(msftMeanAnnually, ibmMeanAnnually))
print('STANDARD DEVIATIONS --')
print('MSFT: %f - IBM: %f' %(msftStandardDeviationAnnually, ibmStandardDeviationAnnually))

# OR WE CAN CREATE AN ARRAY
print('OR --------------------')
means = securityReturns[['MSFT', 'IBM']].mean() * 250
stdDeviations = securityReturns[['MSFT', 'IBM']].std() * 250 ** 0.5
print('MEANS --')
print(means)
print('STANDARD DEVIATIONS --')
print(stdDeviations)


MEANS --
MSFT: 0.263230 - IBM: 0.005184
STANDARD DEVIATIONS --
MSFT: 0.231661 - IBM: 0.205401
OR --------------------
MEANS --
MSFT    0.263230
IBM     0.005184
dtype: float64
STANDARD DEVIATIONS --
MSFT    0.231661
IBM     0.205401
dtype: float64


## Covariance and Correlation

<b>Covariance(returns)</b> : Measures the directional relationship between the returns on two assets. (Ref: https://www.investopedia.com/terms/c/covariance.asp). Reflects the dependence between prices at the different times and focuses on the returns of the portfolio

<b>Correlation(prices)</b> : Focuses on stocl price levels 

Pandas numpy provides variance, covariance and correlation calculations
```python
var = pandas.DataFrame.var()
covarMatrix = pandas.DataFrame.cov() # cov creates a matrix for us
corMatrix = pandas.DataFrame.corr()
```


In [55]:
msftVarAnnually = securityReturns['MSFT'].var() * 250
ibmVarAnnually = securityReturns['IBM'].var() * 250

print('MSFT : %f' % msftVarAnnually)
print('IBM  : %f' % ibmVarAnnually)

MSFT : 0.053667
IBM  : 0.042189


In [60]:
covMatrixAnnually = securityReturns.cov() * 250
covMatrixAnnually

,MSFT,IBM
MSFT,0.053667,0.023148
IBM,0.023148,0.042189


In [64]:
corrMatrix = securityReturns.corr() # we don't need to multiple with 250 (yearly)
corrMatrix

,MSFT,IBM
MSFT,1.000000,0.486479
IBM,0.486479,1.000000


### Portfolio Variance (for 2 stocks)

$$
(w1Q1 + w2Q2)^2 = w1^2Q1^2 + 2w1q1w2q2 + w2^2Q2^2
$$

where
- w1, w2 : weights


## Calculating Portfolio Risk


In [82]:
# assume we have equally weighted portfolio (IBM 50%, MSFT 50%)
weights = np.array([0.5, 0.5])

# portfolio variance
portVar = np.dot(weights.T, np.dot(securityReturns.cov() * 250, weights))
print('Portfolio Variance : %f' % portVar)

# Portfolio Volatility
portVol = (np.dot(weights.T, np.dot(securityReturns.cov() * 250, weights))) ** 0.5
print('Portfolio Volatility : %f%% ' % (portVol * 100))


Portfolio Variance : 0.035538
Portfolio Volatility : 18.851599% 


## Investment Risk

- 1.Un-diversifiable risk (Systematic risk)
  - Depends on the variance of each individual security.
  - Day to day changes in stock prices
  - It impacts the all companies through
    - Economy (recession, etc.)
    - Low consumer spending
    - War / Political instability, etc..
    - Natural disasters
  
- 2.Diversifiable risk (idiosyncratic risk, unsystematic risk)
  - Can be eleminated if we invest in non-correlated assets
    - Automotive + Construction + Energy + Technology
    - Studies show, investing in 20-25 stocks which are not correlated almost removes the Diversifiable risk 
  


### Calculating Diversiable and Non-Diversiable Risk of a Portfolio

$$
diversifiableRisk = portfolioVariance - weightedAnnualVariance
$$

In [87]:
weights = np.array([0.5, 0.5])
print('Weight1 %f  -  Weight2 %f' %(weights[0], weights[1]))

Weight1 0.500000  -  Weight2 0.500000


In [98]:
# Diversiable risk -------------------------

# WRONG CALCULATION --------
# calculate each stocks annual variance
#   We need to convert to float, as dRisk will have vector values 
msftVarAnnual = float(securityReturns[['MSFT']].var() * 250)  # double brackets adds a dimention to the array
ibmVarAnnual = float(securityReturns[['IBM']].var() * 250)  # double brackets adds a dimention to the array

dRisk = portVar - (weights[0]**2 * msftVarAnnual) - (weights[1]**2 * ibmVarAnnual)
print('Diversiable Risk %f%%' %(round(dRisk, 3)*100))



Diversiable Risk 1.200000%


In [104]:
# Non-Diversiable Risk:
ndRisk1 = portVar - dRisk 
ndRisk2 = (weights[0]**2 * msftVarAnnual) + (weights[1]**2 * ibmVarAnnual)

round(ndRisk1,5) == round(ndRisk2,5)


True